In [ ]:
import getpass
import os
import sys

username = getpass.getuser()
location = os.getenv("EXTERNAL_INSTANCE_URL", "")

if username=='neilsen' and location=="https://usdf-rsp.slac.stanford.edu":
    # Modifications of the python path with
    # sys.path do not work if the same namespace
    # is already scanned in the PYTHONPATH.
    # Get rid of the modules pre-loaded so the
    # interpreter reloads lsst.blah.blah.blah
    # after we have made our modifications
    # to sys.path.
    sys.modules.pop("lsst.ts.fbs", None)
    sys.modules.pop("lsst.ts", None)
    sys.modules.pop("lsst", None)
    
    sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.18.1")
    sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/SP-2167/rubin_sim")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_nights")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/lsst_survey_sim")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/ts_fbs_utils/python")

In [ ]:
import getpass
import os
import warnings
import copy
import logging
logging.getLogger().setLevel(logging.INFO)

import numpy as np
import pandas as pd
import sqlite3
import healpy as hp
import matplotlib.pyplot as plt
import colorcet as cc
import skyproj
from IPython.display import display, HTML

from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord
import astropy.units as u

from rubin_scheduler.scheduler.utils import SchemaConverter
from rubin_scheduler.site_models import Almanac
from rubin_scheduler.scheduler.features import Conditions
from rubin_scheduler.utils import ddf_locations, angular_separation, approx_ra_dec2_alt_az, Site, SURVEY_START_MJD

import rubin_sim.maf as maf

from rubin_nights import connections
import rubin_nights.lfa_data as rn_lfa
import rubin_nights.dayobs_utils as rn_dayobs
import rubin_nights.plot_utils as rn_plots
import rubin_nights.augment_visits as augment_visits
import rubin_nights.rubin_scheduler_addons as rn_sch
import rubin_nights.rubin_sim_addons as rn_sim
import rubin_nights.observatory_status as observatory_status
import rubin_nights.scriptqueue as scriptqueue
import rubin_nights.scriptqueue_formatting as scriptqueue_formatting

import importlib

from lsst_survey_sim import lsst_support, simulate_lsst, plot

band_colors = rn_plots.PlotStyles.band_colors
logging.getLogger('lsst_survey_sim').setLevel(logging.INFO)

#%load_ext memory_profiler

In [ ]:
# Set this notebook to run today, for one night, by default.
day_obs = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
sim_nights = 1

survey_start = SURVEY_START_MJD

CONFIG_SCRIPT_PATH = None
tokenfile = None
site = "usdf"
if username=='neilsen' and location=="https://usdf-rsp.slac.stanford.edu":
    CONFIG_SCRIPT_PATH = "/sdf/data/rubin/user/neilsen/devel/ts_config_scheduler/Scheduler/feature_scheduler/maintel/fbs_config_lsst_survey.py"
    CONFIG_DDF_SCRIPT_PATH = "/sdf/data/rubin/user/neilsen/devel/ts_config_scheduler/Scheduler/ddf_gen/lsst_ddf_gen_block_407.py"
    tokenfile = "/home/n/neilsen/.lsst/usdf_access_token"
elif username=='lynnej':
    CONFIG_SCRIPT_PATH = "/Users/lynnej/lsst_repos/ts_config_scheduler/Scheduler/feature_scheduler/maintel/fbs_config_lsst_survey.py"
    CONFIG_DDF_SCRIPT_PATH = "/Users/lynnej/lsst_repos/ts_config_scheduler/Scheduler/ddf_gen/lsst_ddf_gen_block_407.py"
    tokenfile = "/Users/lynnej/.lsst/usdf_rsp"

assert isinstance(CONFIG_SCRIPT_PATH, str), "Please set CONFIG_SCRIPT_PATH"
assert isinstance(tokenfile, str), "Please set tokenfile"


sunset, sunrise = rn_dayobs.day_obs_sunset_sunrise(day_obs, sun_alt=-12)
print(f"DayObs {day_obs}, -12 deg sunset {sunset.iso}, -12 deg sunrise {sunrise.iso}")

In [ ]:
endpoints = connections.get_clients(tokenfile=tokenfile, site=site)
q = endpoints['efd'].select_top_n('lsst.sal.Scheduler.logevent_configurationApplied', ["configurations", "salIndex", "private_origin", "url", "version"], 10, index=1)
ts_commit = q['version'].values[0].replace("heads/", "")
ts_commit

In [ ]:
# Git checkout ts_config_scheduler 
ts_head = simulate_lsst.get_configuration(ts_commit, clone_path='ts_config_scheduler')
ts_head
config_script_path = os.path.join(ts_head, "Scheduler/feature_scheduler/maintel/", "fbs_config_lsst_survey.py")
config_ddf_script_path = os.path.join(ts_head, "Scheduler/ddf_gen/lsst_ddf_gen_block_407.py")

In [ ]:
# Retrieve previous visits to start scheduler 

refresh_visits = True

if refresh_visits:
    initial_opsim = simulate_lsst.fetch_previous_visits(day_obs, tokenfile=tokenfile, site=site)
    if initial_opsim is not None:
        initial_opsim.to_hdf("opsim.h5", key='visits')
else:
    initial_opsim = pd.read_hdf("opsim.h5")
    
if initial_opsim is None:
    print("No visits found")
else:
    print(f"Found {len(initial_opsim)} visits")

In [ ]:
# Configure scheduler and add initial visits

starting_scheduler, initial_opsim, nside = simulate_lsst.setup_scheduler(config_script_path=config_script_path, 
                                                                         config_ddf_script_path=config_ddf_script_path,
                                                                         day_obs=day_obs, 
                                                                         initial_opsim=initial_opsim)

In [ ]:
# np.savez('ddf_obs_wanted.npz', starting_scheduler.survey_lists[2][0].obs_wanted)
# pd.DataFrame(starting_scheduler.survey_lists[2][0].obs_wanted)[['mjd', 'observation_reason', 'observed', 'scheduler_note']]

In [ ]:
# Configure band scheduler
band_scheduler = simulate_lsst.setup_band_scheduler()

# Check what bands it expects for day_obs? 
conditions = Conditions(mjd=sunset.mjd)
band_scheduler(conditions)

In [ ]:
nside = 32
if sim_nights == 1:
    add_downtime = False
    add_clouds = False
else:
    add_downtime = True
    add_clouds = True
starting_observatory, survey_info = simulate_lsst.setup_observatory(day_obs=day_obs, nside=nside, 
                                                                    add_downtime=add_downtime, add_clouds=add_clouds, 
                                                                    seeing=None, real_downtime=False, 
                                                                    initial_opsim=initial_opsim)

In [ ]:
mask = np.where(survey_info['dayobsmjd'] < rn_dayobs.day_obs_to_time(day_obs).mjd + 365)
print(f"Total nighttime {survey_info["hours_in_night"][mask].sum()}, ")
print(f"total downtime {survey_info["downtime_per_night"][mask].sum()}, ")
print(f"available time {survey_info["hours_in_night"][mask].sum() - survey_info["downtime_per_night"][mask].sum()}")
print(f"Average availability mask - {(survey_info["hours_in_night"][mask].sum() - survey_info["downtime_per_night"][mask].sum()) / survey_info["hours_in_night"][mask].sum()}")
print(f"Average availability over survey {survey_info['system_availability']}")

In [ ]:
for start, end in survey_info['downtimes']:
    x = np.floor(start - 0.5)
    plt.plot((x, x), (start - x, end - x) , color='k') 
x = np.floor(survey_info['sunsets12'] - 0.5)
y = survey_info['sunsets12'] - x
plt.fill_between(x, 0.8, y)
x = np.floor(survey_info['sunrises12'] - 0.5)
y = survey_info['sunrises12'] - x
#plt.axvline(rn_dayobs.day_obs_to_time(day_obs).mjd, color='r', linestyle=':', linewidth=1.8)
#plt.axvline(np.floor(Time("2025-09-06T12:00:00").mjd - 0.5))
plt.fill_between(x, y, 1.6)
plt.ylim(0.9, 1.5)
plt.xlim(survey_info['survey_start'].mjd, rn_dayobs.day_obs_to_time(day_obs).mjd + 400)
plt.xlabel("MJD", fontsize='large')
_ = plt.ylabel("Fraction of MJD", fontsize='large')
#plt.savefig("onsky_downtime.png", bbox_inches='tight')

In [ ]:
check_ddfs = True
if check_ddfs:
    ddfs = ddf_locations(skycoords=True)
    lsst_site = Site('LSST')
    times = np.arange(survey_info['survey_start'].mjd, survey_info['survey_end'].mjd, 1/24)
    times = np.arange(np.floor(Time.now().mjd - 0.5),survey_info['survey_end'].mjd, 0.25/24)
    times = np.arange(sunset.mjd - 0.1, sunrise.mjd + 0.1, 0.05/24)
    sunmoon = survey_info['almanac'].get_sun_moon_positions(times)
    moon_ra = sunmoon['moon_RA']
    moon_dec = sunmoon['moon_dec']
    sun_alt = np.degrees(sunmoon['sun_alt'])
    ddf_moon_dist = {}
    ddf_alt = {}
    for ddf in ddfs:
        ddf_moon_dist[ddf] = angular_separation(ddfs[ddf].ra.deg, ddfs[ddf].dec.deg, np.degrees(moon_ra), np.degrees(moon_dec))
        alt, az = approx_ra_dec2_alt_az(ddfs[ddf].ra.deg, ddfs[ddf].dec.deg, lsst_site.latitude, lsst_site.longitude, times, lmst=None)
        ddf_alt[ddf] = alt
    
    plt.figure()
    for ddf in ddfs:
        mask = np.where((sun_alt <= -12) & (ddf_alt[ddf] > 40))
        plt.plot(Time(times[mask], format='mjd').to_datetime(), ddf_moon_dist[ddf][mask], marker='.', linestyle='', label=ddf)
    plt.legend(loc=(1.01, 0.5))
    plt.axvline(sunset.to_datetime(), color='k', linestyle=':')
    plt.axvline(sunrise.to_datetime(), color='k', linestyle=':')
    plt.axhline(30)
    plt.xticks(rotation=90)
    plt.xlabel("MJD")
    plt.ylabel("Distance to moon (deg)")
    
    plt.figure()
    for ddf in ddfs:
        mask = np.where((sun_alt <= -12) & (ddf_alt[ddf] > 40))
        plt.plot(Time(times[mask], format='mjd').to_datetime(), ddf_alt[ddf][mask], marker='.', linestyle='', label=ddf)
    plt.legend(loc=(1.01, 0.5))
    plt.axvline(sunset.to_datetime(), color='k', linestyle=':')
    plt.axvline(sunrise.to_datetime(), color='k', linestyle=':')
    plt.axhline(30)
    plt.xticks(rotation=90)
    plt.xlabel("MJD")
    plt.ylabel("Altitude (deg)")

In [ ]:
reset = True
if reset:
    observatory = copy.deepcopy(starting_observatory)
    scheduler = copy.deepcopy(starting_scheduler)

In [ ]:
# Run the simulation for some nights
observations, scheduler, observatory, rewards, obs_rewards, survey_info = simulate_lsst.run_sim(scheduler=scheduler, 
                                                                                                    band_scheduler=band_scheduler,
                                                                                                    observatory=observatory,
                                                                                                    survey_info=survey_info,
                                                                                                    day_obs=day_obs,
                                                                                                    sim_nights=sim_nights, 
                                                                                                    keep_rewards=False)

In [ ]:
# Need to start at a specific time inside the night?

# sunset, sunrise = rn_dayobs.day_obs_sunset_sunrise(day_obs, sun_alt=-12)
# start = sunset.mjd - 0.1 /24
# end = sunrise.mjd

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore", RuntimeWarning)
#     vals = sim_runner(
#         observatory,
#         scheduler,
#         band_scheduler=fs,
#         sim_start_mjd=start,
#         sim_duration=end-start, 
#         record_rewards=False,
#         verbose=True,
#     )
# observatory = vals[0]
# scheduler = vals[1]
# observations = vals[2]
# if len(vals) == 5:
#     rewards = vals[3]
#     obs_rewards = vals[4]

In [ ]:
filename = "test.db"
# Only new observations?
#obsdf = lsst_support.save_opsim(observatory, observations, initial_opsim=None, filename=filename)
# All observations?
obsdf = lsst_support.save_opsim(observatory, observations, initial_opsim=initial_opsim, filename=filename)

def add_dayobs(x):
    mjdfloor = Time(np.floor(x.observationStartMJD - 0.5) + 0.5, format="mjd", scale="tai")
    return rn_dayobs.day_obs_str_to_int(mjdfloor.isot.split("T")[0])

obsdf["day_obs"] = obsdf.apply(add_dayobs, axis=1)

In [ ]:
if sim_nights == 1:

    targets = pd.DataFrame(observations)
    
    from zoneinfo import ZoneInfo
    tz = ZoneInfo("Chile/Continental")
    tz_utc = ZoneInfo("UTC")
    telescope = "Simonyi"

    site = Site('LSST')
    almanac = Almanac()
    night_events = almanac.get_sunset_info(evening_date=rn_dayobs.day_obs_int_to_str(day_obs), longitude=site.longitude_rad)

    def mjd_to_datetime(mjd, scale='utc', timezone=tz):
        return Time(mjd, format='mjd', scale=scale).utc.to_datetime(timezone=timezone)
        
    eps = 1
    fig, ax = plt.subplots(figsize=(13, 8))
    ax_utc = ax.twiny()
    
    ax.set_title(f"{telescope} DAYOBS {day_obs}", pad=20)
    
    # Shade astronomical events
    ax.fill_between([mjd_to_datetime(night_events['sun_n12_setting']), 
                      mjd_to_datetime(night_events['sun_n18_setting'])],
                     2.5, 0.0, color='lightgray', alpha=0.3)
    ax.fill_between([mjd_to_datetime(night_events['sunset']), 
                     mjd_to_datetime(night_events['sun_n12_setting'])], 
                      2.5, 0.0, color='gray', alpha=0.3)
    ax.fill_between([mjd_to_datetime(night_events['sun_n18_rising']), 
                     mjd_to_datetime(night_events['sun_n12_rising'])],
                     2.5, 0.0, color='lightgray', alpha=0.3)
    ax.fill_between([mjd_to_datetime(night_events['sun_n12_rising']), 
                     mjd_to_datetime(night_events['sunrise'])],
                     2.5, 0.0, color='gray', alpha=0.3)
    # Azimuth constraint period
    ax.fill_between([mjd_to_datetime(night_events['sunrise'] - 3/24),
                    mjd_to_datetime(night_events['sun_n18_rising'])],
                    2.5, 0.0, color='pink', alpha=0.1)
    
    if not np.isnan(night_events['moonrise']):
        ax.axvline(mjd_to_datetime(night_events['moonrise']), linestyle='-', color='blue', alpha=0.3)
    if not np.isnan(night_events['moonset']):
        ax.axvline(mjd_to_datetime(night_events['moonset']), linestyle='-', color='red', alpha=0.3)
    
    colors = cc.glasbey_category10
    # Assign distinct target sets with different colors
    marker_colors = {}
    labels = {}
    count = 0
    if len(targets) > 0:
        for sp in targets.observation_reason.unique():
            marker_colors[sp] = colors[count]
            labels[sp] = sp
            count += 1
    
    if len(targets) > 0:
        visit_alpha = 0.7
        for sp in targets.observation_reason.unique():
            qq = targets.query("observation_reason == @sp")
            label = sp
            ax.plot(mjd_to_datetime(qq.mjd, 'tai'), qq.airmass, 
                    marker='o', linestyle='',
                    color=marker_colors[sp], label=label,
                    alpha=visit_alpha, markerfacecolor='none', zorder=3)
    
    ax.legend(loc=(1.01, 0.0), ncol=2)
    
    x0 = night_events['sunset']+30/60/24
    
    ax.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24), 
             mjd_to_datetime(night_events['sunrise']-30/60/24))
    ax_utc.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24, 'utc', timezone=tz_utc), 
             mjd_to_datetime(night_events['sunrise']-30/60/24, 'utc', timezone=tz_utc))
    
    ax.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24), 
             mjd_to_datetime(night_events['sunrise']-30/60/24))
    ax_utc.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24, 'utc', timezone=tz_utc), 
             mjd_to_datetime(night_events['sunrise']-30/60/24, 'utc', timezone=tz_utc))
    
    # Set ticks relevant sides
    ax.tick_params(axis="x", bottom=True, top=False, labelbottom=True, labeltop=False)
    ax_utc.tick_params(axis="x", bottom=False, top=True, labelbottom=False, labeltop=True)
    
    # Rotate and align bottom ticklabels
    plt.setp([tick.label1 for tick in ax.xaxis.get_major_ticks()], rotation=45,
             ha="right", va="center", rotation_mode="anchor")
    
    # Rotate and align top ticklabels
    plt.setp([tick.label2 for tick in ax_utc.xaxis.get_major_ticks()], rotation=45,
             ha="left", va="center",rotation_mode="anchor")
    
    plt.grid(True, alpha=0.2)
    
    plt.ylim(2.5, 0.9)
    
    ax.set_ylabel("Airmass", fontsize="large")
    ax.set_xlabel(f"Time ({tz})", fontsize="large")
    ax_utc.set_xlabel("Time (UTC)", fontsize='large')
    _ = plt.ylabel("Airmass", fontsize="large")

In [ ]:
# Check all visits have metadata
print(obsdf.target_name.unique())
print(obsdf.science_program.unique())
print(np.sort(obsdf.observation_reason.unique()))

In [ ]:
opsdb = "/Users/lynnej/opsim/fbs_5.1/baseline_v5.1.0_10yrs.db"
conn = sqlite3.connect(opsdb)
baseline_visits = pd.read_sql(f"select * from observations where night <= {obsdf.night.max()}", conn)

In [ ]:
xx = baseline_visits.groupby("observation_reason").agg({'observationStartMJD': "count"}).rename({"observationStartMJD": "baseline_v5.1.0"}, axis=1)
xa = obsdf.groupby("observation_reason").agg({'observationStartMJD': 'count'}).rename({"observationStartMJD": "sim config"}, axis=1)
print(len(baseline_visits), len(obsdf), len(obsdf)/len(baseline_visits))
x = pd.concat([xa, xx], axis=1)
x.loc['total'] = x.sum(axis=0)
x

In [ ]:
ddf_visits = obsdf.query("observation_reason.str.contains('DD') or observation_reason.str.contains('ddf')").copy()
if len(ddf_visits) > 0:
    print("n visits per band")
    ddf_visits.loc[:, 'observation_reason'] = ddf_visits.observation_reason.str.lower()
    ss = ddf_visits.groupby(["observation_reason", "band"]).agg({'observationStartMJD': 'count'})
    ss = ss.reset_index('band').pivot(columns=["band"]).droplevel(0, axis=1)
    ss['all'] = ss.sum(axis=1)
    display(ss.query("observation_reason.str.contains('dd')")[['u', 'g', 'r', 'i', 'z', 'y', 'all']])
    
    print("n days per band")
    ss = ddf_visits.groupby(["observation_reason", "band"]).agg({'day_obs': 'nunique'})
    ss = ss.reset_index('band').pivot(columns=["band"]).droplevel(0, axis=1)
    ss['all'] = ddf_visits.groupby("observation_reason").agg({'day_obs': 'nunique'})
    display(ss.query("observation_reason.str.contains('dd')")[['u', 'g', 'r', 'i', 'z', 'y', 'all']])

In [ ]:
run_calc = True
if run_calc:
    nvisits = {}
    coadd = {}
    m_nvis = maf.CountMetric(col='observationStartMJD', metric_name = "Nvisits")
    m_coadd = maf.Coaddm5Metric(m5_col='fiveSigmaDepth')
    s = maf.HealpixSlicer(nside=64, lon_col='fieldRA', lat_col='fieldDec', rot_sky_pos_col_name = 'rotSkyPos')
    for b in ['u', 'g', 'r', 'i', 'z', 'y', 'all']:
        constraint = f"{b}"
        if b == 'all':
            opsvis = obsdf.to_records()
        else:
            opsvis = obsdf.query("band == @b").to_records()
        nvisits[b] = maf.MetricBundle(m_nvis, s, constraint)
        coadd[b] = maf.MetricBundle(m_coadd, s, constraint)
        g = maf.MetricBundleGroup({f'nvisits {b}': nvisits[b], f'coadd {b}': coadd[b]}, None)
        if len(opsvis) > 0:
            g.run_current(constraint, opsvis)            

In [ ]:
background = plot.get_background(nside=64)
mval = nvisits['all'].metric_values.filled(0)
vmax = np.percentile(nvisits['all'].metric_values.compressed(), 95)
alpha = np.where(np.isnan(background), 0, background)
alpha = np.where(alpha> 0.5, 0.5, alpha)
alpha = np.where(mval > 0, 1, alpha)
plot.hp_moll(mval, alpha=alpha, vmin=0, vmax=vmax)

In [ ]:
background = plot.get_background(nside=64)

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(16, 10),)
axdict = {"u": ax[0][0], "g": ax[0][1], "r": ax[0][2],
          "i": ax[1][0], "z": ax[1][1], "y": ax[1][2], "all": None}
for b in ["u", "g", "r", "i", "z", "y"]:
    if nvisits[b].metric_values is None:
        continue
    if len(nvisits[b].metric_values.compressed()) > 1:
        vmax = np.percentile(nvisits[b].metric_values.compressed(), 95)
    else:
        vmax = None
    label_dec = False
    if b == 'u' or b == 'i':
        label_dec = True
    fig = plot.make_plot(nvisits[b], background=background, proj='McBryde', vmax=vmax, ax=axdict[b], title=f"LSSTCam band {b}", label_dec=label_dec)
fig.tight_layout()
#fig.savefig(os.path.join(out_dir, f"lsstcam_nvisits_band.png"), bbox_inches='tight')

vmax = np.percentile(nvisits['all'].metric_values.compressed(), 95)
fig = plot.make_plot(nvisits['all'], background=background, proj='mcbryde', vmin=None, vmax=vmax, ax=None, title=f"LSSTCam visits")
#fig.savefig(os.path.join(out_dir, f"lsstcam_nvisits.png"), bbox_inches='tight')
fig = plot.make_plot(nvisits['all'], background=background, proj='laea', vmin=None, vmax=vmax, ax=None, title=f"LSSTCam visits")
#fig.savefig(os.path.join(out_dir, f"lsstcam_laea_nvisits.png"), bbox_inches='tight')